# Goal : Scan the 3D predcitions of the FP with the Dpo fold database. 

### A. Run Foldseek
### B. Get the hits with good proba
### C. Report the folds
***

# A

In [18]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
path_pdb = f"{path_project}/FP_pdb"

for file in os.listdir(path_pdb) :
    file_name = file.split("|")[1].split(".pdb")[0]
    print(file_name)
    os.rename(f"{path_pdb}/{file}", f"{path_pdb}/{file_name}.pdb")

NC_022772.1_prot_YP_008771659.1_41
NC_027364.1_prot_YP_009150385.1_71
NC_019526.1_prot_YP_007007686.1_531
NC_023694.1_prot_YP_009012583.1_2
KC821608.1_prot_AGO47494.1_90
NC_017971.2_prot_YP_006382482.1_22
NC_019487.1_prot_YP_007003405.1_148
CP000711.1_prot_ABQ88395.1_14
NC_020883.1_prot_YP_007678101.1_75
AY349011.3_prot_AAQ54997.1_62
NC_006883.2_prot_YP_214361.1_128
NC_027364.1_prot_YP_009150600.1_286
NC_003157.5_prot_NP_438158.1_46
FJ230960.1_prot_ACI90966.1_61
GQ413937.1_prot_ACY66694.1_32
NC_024792.1_prot_YP_009056509.1_240
NC_019400.1_prot_YP_006987067.1_234
NC_019526.1_prot_YP_007007683.1_528
NC_016071.1_prot_YP_004893846.1_39
HQ632825.1_prot_AGN12307.1_149
NC_019414.1_prot_YP_006990136.1_21
NC_019526.1_prot_YP_007007684.1_529
NC_006883.2_prot_YP_214546.1_315
NC_015296.1_prot_YP_004327541.1_170
NC_022772.1_prot_YP_008771658.1_40
FJ230960.1_prot_ACI91001.1_96
NC_025446.1_prot_YP_009101512.1_116
NC_027364.1_prot_YP_009150848.1_534
NC_002730.1_prot_NP_112082.1_49
NC_017971.2_prot_YP_

In [2]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
path_pdb = f"{path_project}/FP_2108"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/FP_2108_foldseek"
    protein_id = path_in.split("/")[-1].split(".pdb")[0].split("'")[1]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/'{file}'" for file in os.listdir(f"{path_pdb}") if file[-3:]=="pdb" if os.path.isfile(f"{path_project}/FP_2108_foldseek/{file.split('.pdb')[0]}.out")== False]

for path in paths :
    seek_beta_helix(path)

***
## B  

> Get a list of the failed Dpos

In [3]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
dir_out = f"{path_project}/FP_2108_foldseek"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

FP_results = {}
no_structure = []
outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.2 :
                FP_results[row["query"]]= (row["prob"],row["target"] , fold)
                break
    else :
        no_structure.append(row["query"])
                


In [4]:
import pprint
pp = pprint.PrettyPrinter(width = 150, sort_dicts = True, compact = False)
pp.pprint(FP_results)




{'33__FP__T12.pdb': (1.0, 'AF-A0A1E5LEG1-F1-model_v4.pdb', 'right-handed beta-helix'),
 '36__FP__T12.pdb': (0.998, 'K17alfa62__cds_64_A_3_129_548.pdb', 'right-handed beta-helix'),
 '67__FP__T12.pdb': (0.855, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 '6__FP__T12.pdb': (1.0, 'AF-A0A1E5LEG1-F1-model_v4.pdb', 'right-handed beta-helix'),
 '79__FP__T12.pdb': (0.996, 'AF-A0A496B6X2-F1-model_v4.pdb', '6-bladed beta-propeller'),
 '97__FP__T12.pdb': (0.353, 'AF-A0A7G2DEW9-F1-model_v4.pdb', '6-bladed beta-propeller'),
 '99__FP__T12.pdb': (0.923, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'FQ482084.1_prot_CBX44498.1_37.pdb': (0.632, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'GQ413937.1_prot_ACY66694.1_32.pdb': (0.353, 'AF-A0A654UB90-F1-model_v4.pdb', '6-bladed beta-propeller'),
 'GQ413937.1_prot_ACY66700.1_38.pdb': (1.0, 'K17alfa62__cds_64_A_3_129_548.pdb', 'right-handed beta-helix'),
 'HQ632825.1_prot_AGN12247.1_89.pdb': (1.0, 'K39PH122C2

In [7]:
len(no_structure) , len(FP_results)

(20, 65)

In [1]:
import nglview as nv

def view_pdb(file) : 
    path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
    path_pdb = f"{path_project}/FP_2108"
    view = nv.show_structure_file(f"{path_pdb}/{file}")
    view


In [2]:
path_project = "/media/concha-eloko/Linux/PhageDEPOdetection"
path_pdb = f"{path_project}/FP_pdb"

view = nv.show_file(f"{path_pdb}/KC821625.1_prot_AGO49026.1_15.pdb")
view.background = 'black'
view

NGLWidget(background='black')

In [28]:
no_structure

['NC_017972.1_prot_YP_006382630.1_100.pdb',
 'NC_016071.1_prot_YP_004893846.1_39.pdb',
 'KC821625.1_prot_AGO49026.1_15.pdb',
 'NC_019401.1_prot_YP_006987325.1_223.pdb',
 'NC_027364.1_prot_YP_009150318.1_4.pdb',
 'NC_019526.1_prot_YP_007007689.1_534.pdb',
 'NC_019929.1_prot_YP_007237854.1_204.pdb',
 'NC_006883.2_prot_YP_214266.1_30.pdb',
 'NC_025422.1_prot_YP_009098968.1_210.pdb',
 'NC_027364.1_prot_YP_009150602.1_288.pdb',
 'KC821621.1_prot_AGO48656.1_21.pdb',
 'HQ632825.1_prot_AGN12459.1_301.pdb',
 'NC_027364.1_prot_YP_009150315.1_1.pdb',
 'NC_017971.2_prot_YP_006382482.1_22.pdb',
 'JX262376.1_prot_AFO10887.1_21.pdb',
 'AY349011.3_prot_AAQ55000.1_65.pdb',
 'NC_006883.2_prot_YP_214361.1_128.pdb',
 'NC_019526.1_prot_YP_007007233.1_78.pdb',
 'NC_023557.1_prot_YP_009004812.1_20.pdb',
 'NC_003157.5_prot_NP_438158.1_46.pdb',
 'NC_019917.1_prot_YP_007236806.1_60.pdb',
 'HQ632825.1_prot_AGN12302.1_144.pdb',
 'NC_020079.1_prot_YP_007348548.1_180.pdb',
 'NC_027364.1_prot_YP_009150606.1_292.pdb'

In [ ]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/depolymerase_building"
path_pdb = f"{path_project}/phagedepo_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/CAZY_seekfold"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/{file}" for file in os.listdir(f"{path_pdb}") if file not in first_batch_dpos]


for path in paths :
    seek_beta_helix(path)

***
## C.

In [7]:
pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)

pp.pprint(decipher_phagedepo)

{'1043.pdb': ('AF-A0A202B6A7-F1-model_v4.pdb', 0.353),
 '1125.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.353),
 '1574.pdb': ('A1l_00058_A_1_129_274.pdb', 0.473),
 '1810.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.411),
 '1842.pdb': ('AF-A0A7K3HGV9-F1-model_v4.pdb', 0.353),
 '1873.pdb': ('AF-A0A0F6AK51-F1-model_v4.pdb', 0.277),
 '1918.pdb': ('AF-A0A0D3A0D6-F1-model_v4.pdb', 0.326),
 '2001.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.442),
 '2048.pdb': ('A1l_00058_A_1_129_274.pdb', 0.411),
 '2170.pdb': ('7fhv.pdb_B', 0.277),
 '2190.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.382),
 '2217.pdb': ('AF-A0A1E5LEG1-F1-model_v4.pdb', 0.473),
 '2588.pdb': ('AF-A0A7W9JCS6-F1-model_v4_A_1_95_273.pdb', 0.473),
 '2617.pdb': ('AF-A0A562WF65-F1-model_v4.pdb', 0.326),
 '2641.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0.254),
 '2691.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.442),
 '2753.pdb': ('AF-A0A7C4BP91-F1-model_v4.pdb', 0.473),
 '2786.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0